## Setup

In [ ]:
import sys
import os
import IPython
import multiprocessing
import copy
import pickle
import warnings
from datetime import datetime
from time import time
from matplotlib import font_manager as fm, rc, rcParams
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from numpy import array, nan, random as rnd, where as which
import pandas as pd
from pandas import DataFrame as dataframe, Series as series, isna, isnull, read_csv
from pandas.tseries.offsets import DateOffset
from scipy.special import boxcox1p
from scipy.stats import skew

import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr

from sklearn.linear_model import LinearRegression
import xgboost as xgb
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tqdm import tqdm

# display setting
warnings.filterwarnings(action='ignore')
rcParams['axes.unicode_minus'] = False
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 400)
pd.set_option('display.width', 1500)

# plot setting
# 폰트 경로 본인 PC에 맞춰 설정
font_path = 'c:/windows/fonts/KoPub Dotum Light.ttf'
font_obj = fm.FontProperties(fname=font_path, size=12).get_name()
rc('font', family=font_obj)
myColors = sns.hls_palette(20, s=0.4)

## Loading data

In [ ]:
#기본 세팅 및 파일 불러오기
# Get Stock List
path = 'C:/Users/nashv/Desktop/asiaE_FinalProject/DACON/open_week4'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission_week4.csv'

# Get Data & Modeling
start_date = '20210104'
end_date = '20210917'

In [ ]:
# path와 list_name을 조인하여 csv 파일 불러옴
stock_list = pd.read_csv(os.path.join(path ,list_name))
# 종목코드를 6자리로 맞추고 부족하면 앞에 0으로 채움
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))

# 시작 날짜를 weekday() 메서드를 사용하여 Python에서 요일 이름 가져 옴
start_weekday = pd.to_datetime(start_date).weekday()
# 마지막 날짜를 53주 내에서 몇주차 인지 전환
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date ,end_date ,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

sample_name = 'sample_submission_week4.csv'
sample_submission = pd.read_csv(os.path.join(path ,sample_name))

## Define Model & Training

### 1. LightGBM

In [ ]:
# LGBM 3주차
model = LGBMRegressor()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')

    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 8월 13일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 8월 20일까지의 데이터를 학습한다.
    y_0 = y[: ,0]
    y_1 = y[: ,1]
    y_2 = y[: ,2]
    y_3 = y[: ,3]
    y_4 = y[: ,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 8월 23일부터 8월 27일까지의 데이터를 예측할 것이다.

    predictions = []
    for y_value in y_values :
        model.fit(x ,y_value)
        prediction = model.predict(np.expand_dims(x_public ,0))
        predictions.append(prediction[0])
    sample_submission.loc[: ,code] = predictions * 2
sample_submission.isna().sum().sum()

columns = list(sample_submission.columns[1:])
columns = ['Day'] + [str(x).zfill(6) for x in columns]
sample_submission.columns = columns
sample_submission.to_csv('BASELINE_LGBMRegressor_Week4.csv' ,index=False)
sample_submission

### 2. XGBoost

In [ ]:
# xgboost 3추자
model = XGBRegressor()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')

    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 8월 13일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 8월 20일까지의 데이터를 학습한다.
    y_0 = y[: ,0]
    y_1 = y[: ,1]
    y_2 = y[: ,2]
    y_3 = y[: ,3]
    y_4 = y[: ,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 8월 23일부터 8월 27일까지의 데이터를 예측할 것이다.

    predictions = []
    for y_value in y_values :
        model.fit(x ,y_value)
        prediction = model.predict(np.expand_dims(x_public ,0))
        predictions.append(prediction[0])
    sample_submission.loc[: ,code] = predictions * 2
sample_submission.isna().sum().sum()

columns = list(sample_submission.columns[1:])
columns = ['Day'] + [str(x).zfill(6) for x in columns]
sample_submission.columns = columns
sample_submission.to_csv('BASELINE_XGBRegressor_Week4.csv' ,index=False)
sample_submission

### 3. CatBoost

In [ ]:
# CatBoost 3주차
model = CatBoostRegressor()
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')

    x = data.iloc[0:-2].to_numpy() # 2021년 1월 04일 ~ 2021년 8월 13일까지의 데이터로
    y = data.iloc[1:-1].to_numpy() # 2021년 1월 11일 ~ 2021년 8월 20일까지의 데이터를 학습한다.
    y_0 = y[: ,0]
    y_1 = y[: ,1]
    y_2 = y[: ,2]
    y_3 = y[: ,3]
    y_4 = y[: ,4]

    y_values = [y_0, y_1, y_2, y_3, y_4]
    x_public = data.iloc[-2].to_numpy() # 2021년 8월 23일부터 8월 27일까지의 데이터를 예측할 것이다.

    predictions = []
    for y_value in y_values :
        model.fit(x ,y_value)
        prediction = model.predict(np.expand_dims(x_public ,0))
        predictions.append(prediction[0])
    sample_submission.loc[: ,code] = predictions * 2
sample_submission.isna().sum().sum()

columns = list(sample_submission.columns[1:])
columns = ['Day'] + [str(x).zfill(6) for x in columns]
sample_submission.columns = columns
sample_submission.to_csv('BASELINE_CatBoostRegressor_Week4.csv' ,index=False)
sample_submission